# Transaction Data Analysis

This notebook analyzes transaction patterns and creates a graph structure for money laundering detection.

In [1]:
!pip install polars
!pip install networkx
!pip install matplotlib
!pip install igraph

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   -------------------------- ------------- 2.1/3.2 MB 18.9 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 18.2 MB/s  0:00:00

   -------------------- ------------------- 1/2 [igraph]
   ---------------------------------------- 2/2 [igraph]



## Setup

Install required library.

In [2]:
import polars as pl
import networkx as nx
import matplotlib.pyplot as plt

# df = pl.read_csv('data/HI-Small_Trans.csv')
# Try lazy frame
df = pl.read_csv('data/HI-Small_Trans.csv')

# Sample only 10% of full data for memory management, commented out if want full data
df = df.sample(fraction=0.10, with_replacement=False, seed=42).lazy()

## Load Data

Read transaction data from CSV file.

In [3]:
df.collect()

Timestamp,From Bank,Account,To Bank,Account_duplicated_0,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
str,i64,str,i64,str,f64,str,f64,str,str,i64
"""2022/09/01 14:51""",122332,"""808376670""",220504,"""808763050""",165.58,"""UK Pound""",165.58,"""UK Pound""","""Cash""",0
"""2022/09/02 12:05""",70,"""100428660""",2843,"""800C08F10""",1000.0,"""US Dollar""",1000.0,"""US Dollar""","""Cheque""",0
"""2022/09/07 22:23""",14,"""8035A5A50""",3,"""8035CB140""",4449.0,"""Yuan""",4449.0,"""Yuan""","""Credit Card""",0
"""2022/09/02 15:44""",14290,"""8028B20E0""",214100,"""80B5A8EF0""",24022.22,"""Euro""",24022.22,"""Euro""","""ACH""",0
"""2022/09/01 00:28""",12735,"""803109B90""",226951,"""80A633740""",2777.01,"""US Dollar""",2777.01,"""US Dollar""","""Cheque""",0
…,…,…,…,…,…,…,…,…,…,…
"""2022/09/06 20:48""",124,"""813B87B71""",124,"""813B87B71""",0.026602,"""Bitcoin""",0.026602,"""Bitcoin""","""Bitcoin""",0
"""2022/09/02 01:05""",70,"""1004286A8""",111141,"""8144DAE90""",1614.02,"""Euro""",1614.02,"""Euro""","""Cash""",0
"""2022/09/09 12:46""",12381,"""8085123D0""",216645,"""80F2F2D50""",320.56,"""US Dollar""",320.56,"""US Dollar""","""Credit Card""",0


In [4]:
df = df.with_columns(
    pl.col('Timestamp').str.strptime(pl.Datetime, format='%Y/%m/%d %H:%M')
)

## Data Preparation

Convert timestamp column to datetime format.

In [5]:
df.collect()

Timestamp,From Bank,Account,To Bank,Account_duplicated_0,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
datetime[μs],i64,str,i64,str,f64,str,f64,str,str,i64
2022-09-01 14:51:00,122332,"""808376670""",220504,"""808763050""",165.58,"""UK Pound""",165.58,"""UK Pound""","""Cash""",0
2022-09-02 12:05:00,70,"""100428660""",2843,"""800C08F10""",1000.0,"""US Dollar""",1000.0,"""US Dollar""","""Cheque""",0
2022-09-07 22:23:00,14,"""8035A5A50""",3,"""8035CB140""",4449.0,"""Yuan""",4449.0,"""Yuan""","""Credit Card""",0
2022-09-02 15:44:00,14290,"""8028B20E0""",214100,"""80B5A8EF0""",24022.22,"""Euro""",24022.22,"""Euro""","""ACH""",0
2022-09-01 00:28:00,12735,"""803109B90""",226951,"""80A633740""",2777.01,"""US Dollar""",2777.01,"""US Dollar""","""Cheque""",0
…,…,…,…,…,…,…,…,…,…,…
2022-09-06 20:48:00,124,"""813B87B71""",124,"""813B87B71""",0.026602,"""Bitcoin""",0.026602,"""Bitcoin""","""Bitcoin""",0
2022-09-02 01:05:00,70,"""1004286A8""",111141,"""8144DAE90""",1614.02,"""Euro""",1614.02,"""Euro""","""Cash""",0
2022-09-09 12:46:00,12381,"""8085123D0""",216645,"""80F2F2D50""",320.56,"""US Dollar""",320.56,"""US Dollar""","""Credit Card""",0


Disregard transaction with Payment Format "Reinvesment"

In [6]:
df = df.filter(pl.col("Payment Format") != "Reinvestment")

## Create Nodes

Build graph nodes from transactions with ID, sender, receiver, time, amount, and label.

In [7]:
nodes = df.with_row_index("node_id").select([
    pl.col("node_id"),
    pl.col("Account").alias("f_i"),                # From
    pl.col("Account_duplicated_0").alias("b_i"),   # Beneficiary
    pl.col("Timestamp").alias("t_i"),              # Time
    pl.col("Amount Received").alias("a_i"),        # Amount
    pl.col("Is Laundering")                        # Ground truth
])

In [8]:
nodes.collect()

node_id,f_i,b_i,t_i,a_i,Is Laundering
u32,str,str,datetime[μs],f64,i64
0,"""808376670""","""808763050""",2022-09-01 14:51:00,165.58,0
1,"""100428660""","""800C08F10""",2022-09-02 12:05:00,1000.0,0
2,"""8035A5A50""","""8035CB140""",2022-09-07 22:23:00,4449.0,0
3,"""8028B20E0""","""80B5A8EF0""",2022-09-02 15:44:00,24022.22,0
4,"""803109B90""","""80A633740""",2022-09-01 00:28:00,2777.01,0
…,…,…,…,…,…
459751,"""813B87B71""","""813B87B71""",2022-09-06 20:48:00,0.026602,0
459752,"""1004286A8""","""8144DAE90""",2022-09-02 01:05:00,1614.02,0
459753,"""8085123D0""","""80F2F2D50""",2022-09-09 12:46:00,320.56,0


## Create Edges

Connect transactions where one receiver becomes the sender in another transaction.

In [9]:
edges = nodes.join(
    nodes,
    left_on="b_i", 
    right_on="f_i",
    suffix="_d"
).rename({"node_id": "v_s", "node_id_d": "v_d"})

In [10]:
edges.collect()

v_s,f_i,b_i,t_i,a_i,Is Laundering,v_d,b_i_d,t_i_d,a_i_d,Is Laundering_d
u32,str,str,datetime[μs],f64,i64,u32,str,datetime[μs],f64,i64
99295,"""100428810""","""808376670""",2022-09-10 12:26:00,94.14,0,0,"""808763050""",2022-09-01 14:51:00,165.58,0
225193,"""8076829F0""","""808376670""",2022-09-02 18:32:00,1537.16,0,0,"""808763050""",2022-09-01 14:51:00,165.58,0
280904,"""100428810""","""808376670""",2022-09-04 08:28:00,55.19,0,0,"""808763050""",2022-09-01 14:51:00,165.58,0
300146,"""100428810""","""808376670""",2022-09-02 01:56:00,55.19,0,0,"""808763050""",2022-09-01 14:51:00,165.58,0
13375,"""80571C290""","""100428660""",2022-09-09 06:35:00,529.2,0,1,"""800C08F10""",2022-09-02 12:05:00,1000.0,0
…,…,…,…,…,…,…,…,…,…,…
451074,"""805836AE0""","""1004286A8""",2022-09-09 16:27:00,3503.46,0,459752,"""8144DAE90""",2022-09-02 01:05:00,1614.02,0
454084,"""803BA4C60""","""1004286A8""",2022-09-02 13:23:00,1370.04,0,459752,"""8144DAE90""",2022-09-02 01:05:00,1614.02,0
69627,"""801F49CC0""","""8085123D0""",2022-09-08 00:56:00,1543.08,0,459753,"""80F2F2D50""",2022-09-09 12:46:00,320.56,0


## Time Delta Window 

In [11]:
timedelta = pl.duration(hours=24)

## Filter Edges

Keep only edges where the second transaction occurs within 24 hours after the first.

In [12]:
edges = edges.filter(
    (pl.col("t_i_d") > pl.col("t_i")) & 
    (pl.col("t_i_d") < pl.col("t_i") + timedelta)
)

## Temporal View Results

Display final nodes and edges.

In [13]:
print(nodes.collect())

shape: (459_756, 6)
┌─────────┬───────────┬───────────┬─────────────────────┬──────────┬───────────────┐
│ node_id ┆ f_i       ┆ b_i       ┆ t_i                 ┆ a_i      ┆ Is Laundering │
│ ---     ┆ ---       ┆ ---       ┆ ---                 ┆ ---      ┆ ---           │
│ u32     ┆ str       ┆ str       ┆ datetime[μs]        ┆ f64      ┆ i64           │
╞═════════╪═══════════╪═══════════╪═════════════════════╪══════════╪═══════════════╡
│ 0       ┆ 808376670 ┆ 808763050 ┆ 2022-09-01 14:51:00 ┆ 165.58   ┆ 0             │
│ 1       ┆ 100428660 ┆ 800C08F10 ┆ 2022-09-02 12:05:00 ┆ 1000.0   ┆ 0             │
│ 2       ┆ 8035A5A50 ┆ 8035CB140 ┆ 2022-09-07 22:23:00 ┆ 4449.0   ┆ 0             │
│ 3       ┆ 8028B20E0 ┆ 80B5A8EF0 ┆ 2022-09-02 15:44:00 ┆ 24022.22 ┆ 0             │
│ 4       ┆ 803109B90 ┆ 80A633740 ┆ 2022-09-01 00:28:00 ┆ 2777.01  ┆ 0             │
│ …       ┆ …         ┆ …         ┆ …                   ┆ …        ┆ …             │
│ 459751  ┆ 813B87B71 ┆ 813B87B71 ┆ 2022-09-0

## Second Order Graph Creation

### Edge creation

In [14]:
s_edges = edges.with_columns([
    pl.concat_str([pl.col("f_i"), pl.col("b_i")], separator="->").alias("v_s"),
    pl.concat_str([pl.col("b_i"), pl.col("b_i_d")], separator="->").alias("v_d")
]).select(["v_s", "v_d"])

In [15]:
s_edges.collect().head()

v_s,v_d
str,str
"""81269FE50->100428660""","""100428660->800C08F10"""
"""8070EAE50->100428660""","""100428660->800C08F10"""
"""806018480->100428660""","""100428660->800C08F10"""
"""80D167C20->100428660""","""100428660->800C08F10"""
"""80C710A10->100428660""","""100428660->800C08F10"""


### Weight Calculation

In [16]:
# Count times where one node goes to another
s_edges_with_count = s_edges.group_by(["v_s", "v_d"]).agg([
    pl.len().alias("spec_count")
])

denom_P = s_edges.group_by("v_s").agg([
    pl.len().alias("s_to_any")
])

denom_P_prime = s_edges.group_by("v_d").agg([
    pl.len().alias("any_to_d")
])

In [17]:
print(s_edges_with_count.collect().head())

print(denom_P.collect().head())

print(denom_P_prime.collect().head())

shape: (5, 3)
┌──────────────────────┬──────────────────────┬────────────┐
│ v_s                  ┆ v_d                  ┆ spec_count │
│ ---                  ┆ ---                  ┆ ---        │
│ str                  ┆ str                  ┆ u32        │
╞══════════════════════╪══════════════════════╪════════════╡
│ 8082338C0->100428660 ┆ 100428660->809057420 ┆ 1          │
│ 80D06B000->100428660 ┆ 100428660->8118B7910 ┆ 1          │
│ 800385230->1004286A8 ┆ 1004286A8->8052E3030 ┆ 2          │
│ 80D167C20->100428660 ┆ 100428660->807CBC910 ┆ 1          │
│ 800068890->8011FA410 ┆ 8011FA410->8015C29D0 ┆ 1          │
└──────────────────────┴──────────────────────┴────────────┘
shape: (5, 2)
┌──────────────────────┬──────────┐
│ v_s                  ┆ s_to_any │
│ ---                  ┆ ---      │
│ str                  ┆ u32      │
╞══════════════════════╪══════════╡
│ 8061C85B0->8065DFD90 ┆ 2        │
│ 801B94FE0->802814A40 ┆ 1        │
│ 80EECD850->80F150A70 ┆ 1        │
│ 8030A4470->

In [18]:
s_edges_with_weight = s_edges_with_count.join(
    denom_P,
    on="v_s",
    how="left"
).join(
    denom_P_prime,
    on="v_d",
    how="left"
).with_columns([
    (pl.col("spec_count") / pl.col("s_to_any")).alias("P"),
    (pl.col("spec_count") / pl.col("any_to_d")).alias("P_prime")
]).with_columns([
    (pl.max_horizontal([pl.col("P"), pl.col("P_prime")])).alias("weight")
])

In [19]:
print(s_edges_with_weight.collect().head())

shape: (5, 8)
┌───────────────┬──────────────┬────────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ v_s           ┆ v_d          ┆ spec_count ┆ s_to_any ┆ any_to_d ┆ P        ┆ P_prime  ┆ weight   │
│ ---           ┆ ---          ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str           ┆ str          ┆ u32        ┆ u32      ┆ u32      ┆ f64      ┆ f64      ┆ f64      │
╞═══════════════╪══════════════╪════════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 8070EAE50->10 ┆ 100428660->8 ┆ 1          ┆ 1876     ┆ 25       ┆ 0.000533 ┆ 0.04     ┆ 0.04     │
│ 0428660       ┆ 01A57120     ┆            ┆          ┆          ┆          ┆          ┆          │
│ 100428858->80 ┆ 80814A080->8 ┆ 1          ┆ 2        ┆ 1        ┆ 0.5      ┆ 1.0      ┆ 1.0      │
│ 814A080       ┆ 094A3D50     ┆            ┆          ┆          ┆          ┆          ┆          │
│ 810F370A0->10 ┆ 1004286F0->8 ┆ 1          ┆ 226      ┆ 3        ┆ 0.004425 

### Apply second order graph's weight to Temporal graph's edges

In [20]:
edges_with_weight = edges.with_columns([
    pl.concat_str([pl.col("f_i"), pl.col("b_i")], separator="->").alias("tx_s"),
    pl.concat_str([pl.col("b_i"), pl.col("b_i_d")], separator="->").alias("tx_d")
]).join(
    s_edges_with_weight.select([
        "v_s", "v_d", "weight"
    ]),
    left_on=["tx_s", "tx_d"],
    right_on=["v_s", "v_d"],
    how="left"
).drop([
    "tx_s", "tx_d", "v_s", "v_d"
])

In [21]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 1)

f_i,b_i,t_i,a_i,Is Laundering,b_i_d,t_i_d,a_i_d,Is Laundering_d,weight
str,str,datetime[μs],f64,i64,str,datetime[μs],f64,i64,f64
"""100428660""","""800704A30""",2022-09-06 16:52:00,7093.06,1,"""809031710""",2022-09-07 09:52:00,212.95,0,1.0
"""80AEB33D0""","""80AEB4990""",2022-09-04 13:26:00,862.64,1,"""80AEB4990""",2022-09-05 13:22:00,4.24,0,0.666667
"""80018E990""","""800059C00""",2022-09-08 13:14:00,6742.83,1,"""80018E990""",2022-09-08 14:22:00,1518.21,0,0.5
"""8001BB380""","""8001A6870""",2022-09-01 17:48:00,1499.02,1,"""8016C6620""",2022-09-01 20:46:00,29.05,0,1.0
"""804B56250""","""807C35490""",2022-09-06 14:55:00,4445.55,1,"""808B09840""",2022-09-07 12:53:00,1223.72,0,1.0
…,…,…,…,…,…,…,…,…,…
"""8021353D0""","""80266F880""",2022-09-09 09:47:00,14368.04,1,"""80286DE40""",2022-09-10 02:27:00,9783.49,0,0.5
"""8061A6010""","""806C62690""",2022-09-06 13:24:00,832531.23,1,"""8073A2C80""",2022-09-07 04:14:00,3.6226e6,0,1.0
"""80777E5C0""","""800E3D880""",2022-09-06 17:05:00,4264.34,1,"""808173600""",2022-09-07 06:36:00,1135.38,0,1.0


In [22]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 1)

f_i,b_i,t_i,a_i,Is Laundering,b_i_d,t_i_d,a_i_d,Is Laundering_d,weight
str,str,datetime[μs],f64,i64,str,datetime[μs],f64,i64,f64
"""100428660""","""800704A30""",2022-09-06 16:52:00,7093.06,1,"""809031710""",2022-09-07 09:52:00,212.95,0,1.0
"""80AEB33D0""","""80AEB4990""",2022-09-04 13:26:00,862.64,1,"""80AEB4990""",2022-09-05 13:22:00,4.24,0,0.666667
"""80018E990""","""800059C00""",2022-09-08 13:14:00,6742.83,1,"""80018E990""",2022-09-08 14:22:00,1518.21,0,0.5
"""8001BB380""","""8001A6870""",2022-09-01 17:48:00,1499.02,1,"""8016C6620""",2022-09-01 20:46:00,29.05,0,1.0
"""804B56250""","""807C35490""",2022-09-06 14:55:00,4445.55,1,"""808B09840""",2022-09-07 12:53:00,1223.72,0,1.0
…,…,…,…,…,…,…,…,…,…
"""8021353D0""","""80266F880""",2022-09-09 09:47:00,14368.04,1,"""80286DE40""",2022-09-10 02:27:00,9783.49,0,0.5
"""8061A6010""","""806C62690""",2022-09-06 13:24:00,832531.23,1,"""8073A2C80""",2022-09-07 04:14:00,3.6226e6,0,1.0
"""80777E5C0""","""800E3D880""",2022-09-06 17:05:00,4264.34,1,"""808173600""",2022-09-07 06:36:00,1135.38,0,1.0


In [23]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 1)["weight"].value_counts().sort("weight")

weight,count
f64,u32
0.333333,1
0.5,9
0.666667,2
1.0,84


In [24]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 0)["weight"].value_counts().sort("weight")

weight,count
f64,u32
0.004202,20
0.004329,12
0.00463,18
0.005155,63
0.005319,25
…,…
0.857143,66
0.875,21
0.888889,8


In [25]:
df_for_leiden = edges_with_weight.select([
    pl.concat_str(["f_i", "b_i", "t_i"]).alias("source_node"),
    pl.concat_str(["b_i", "b_i_d", "t_i_d"]).alias("target_node"),
    pl.col("weight")
])

In [26]:
df_for_leiden.collect()

source_node,target_node,weight
str,str,f64
"""81269FE501004286602022-09-02 0…","""100428660800C08F102022-09-02 1…",0.041667
"""8070EAE501004286602022-09-02 1…","""100428660800C08F102022-09-02 1…",0.041667
"""8060184801004286602022-09-02 0…","""100428660800C08F102022-09-02 1…",0.041667
"""80D167C201004286602022-09-02 0…","""100428660800C08F102022-09-02 1…",0.041667
"""80C710A101004286602022-09-02 0…","""100428660800C08F102022-09-02 1…",0.041667
…,…,…
"""8137529A1813B87B712022-09-06 0…","""813B87B71813B87B712022-09-06 2…",1.0
"""8093BB6401004286A82022-09-02 0…","""1004286A88144DAE902022-09-02 0…",0.25
"""80E2EFCF01004286A82022-09-02 0…","""1004286A88144DAE902022-09-02 0…",0.25


In [31]:
df_for_leiden_with_weight_filtered = df_for_leiden.filter(pl.col("weight") >= 0.5)

In [32]:
df_for_leiden_with_weight_filtered.collect()

source_node,target_node,weight
str,str,f64
"""8001A84608001E5B002022-09-08 0…","""8001E5B00800A36BC02022-09-08 1…",1.0
"""80979071080F2938C02022-09-08 1…","""80F2938C08104648502022-09-09 0…",1.0
"""800284530800798B002022-09-06 1…","""800798B008009E68A02022-09-07 1…",1.0
"""8005DBC308017C91C02022-09-07 1…","""8017C91C0802E2DAE02022-09-08 1…",1.0
"""8090F74E08091977702022-09-01 1…","""80919777080924BA002022-09-02 0…",1.0
…,…,…
"""80E1A3C5080E23FCD02022-09-01 1…","""80E23FCD080E7EB7E02022-09-02 1…",1.0
"""80E868AD080F1238A02022-09-01 2…","""80F1238A080F479BB02022-09-02 0…",1.0
"""80E868AD080F1238A02022-09-02 0…","""80F1238A080F479BB02022-09-02 0…",1.0


In [33]:
import igraph as ig

In [34]:
edges_data = df_for_leiden_with_weight_filtered.select([
    "source_node", "target_node", "weight"
]).collect().to_numpy()

In [38]:
g = ig.Graph.TupleList(edges_data, directed=True, weights=True)

In [39]:
partition = g.community_leiden(weights='weight', objective_function='modularity')

In [40]:
partition

In [43]:
# Create a mapping of Transaction ID -> Community ID
node_names = g.vs["name"]
community_ids = partition.membership

community_df = pl.DataFrame({
    "node_id_str": node_names,
    "community_id": community_ids
})

# Now you can join this back to your original transaction data to see the "Fraud Rings"

In [44]:
community_df

node_id_str,community_id
str,i64
"""8001A84608001E5B002022-09-08 0…",0
"""8001E5B00800A36BC02022-09-08 1…",0
"""80979071080F2938C02022-09-08 1…",1
"""80F2938C08104648502022-09-09 0…",1
"""800284530800798B002022-09-06 1…",2
…,…
"""80E868AD080F1238A02022-09-01 2…",602
"""80F1238A080F479BB02022-09-02 0…",602
"""80E868AD080F1238A02022-09-02 0…",602
